# Integrate an orbit and compute uncertainties in Milky Way potential model

`gala` provides a simple mass model for the Milky Way based on recent measurements of the enclosed mass compiled from the literature. See the [Defining a Milky Way potential model](../potential/define-milky-way-model.html) documentation for more information about how this model was defined.

In this example, we'll use the position and velocity and uncertainties of the Milky Way satellite galaxy "Draco" to integrate orbits in a Milky Way mass model starting from samples from the error distribution over initial conditions defined by its observed kinematics. We'll then compute distributions of orbital properties like orbital period, pericenter, and eccentricity.

In [ ]:
# Some imports we'll need later:

# Third-party
import astropy.units as u
import astropy.coordinates as coord
from astropy.io import ascii
import matplotlib.pyplot as plt
import numpy as np

# Gala
from gala.mpl_style import mpl_style
plt.style.use(mpl_style)
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic
%matplotlib inline

For the sky position and distance of Draco, we'll use measurements from [Bonanos et al. 2004](https://arxiv.org/abs/astro-ph/0310477). For proper motion components, we'll use the recent HSTPROMO measurements ([Sohn et al. 2017](https://arxiv.org/abs/1707.02593)) and the line-of-sight velocity from [Walker et al. 2007](https://arxiv.org/abs/0708.0010).

In [ ]:
icrs = coord.ICRS(ra=coord.Angle('17h 20m 12.4s'), 
                  dec=coord.Angle('+57° 54′ 55″'),
                  distance=76*u.kpc,
                  pm_ra_cosdec=0.0569*u.mas/u.yr,
                  pm_dec=-0.1673*u.mas/u.yr,
                  radial_velocity=-291*u.km/u.s)

icrs_err = coord.ICRS(ra=0*u.deg, dec=0*u.deg, distance=6*u.kpc,
                      pm_ra_cosdec=0.009*u.mas/u.yr,
                      pm_dec=0.009*u.mas/u.yr,
                      radial_velocity=0.1*u.km/u.s)

Let's start by transforming the measured values to a Galactocentric reference frame so we can integrate an orbit in our Milky Way model:

In [ ]:
gc_frame = coord.Galactocentric(galcen_distance=8.3*u.kpc,
                                z_sun=0*u.pc,
                                galcen_v_sun=coord.CartesianDifferential([11.1, 250, 7.25]*u.km/u.s))

gc = icrs.transform_to(gc_frame)

In [ ]:
w0 = gd.PhaseSpacePosition(gc.data)

In [ ]:
potential = gp.MilkyWayPotential()

In [ ]:
orbit = potential.integrate_orbit(w0, dt=-0.5, n_steps=8000)

In [ ]:
_ = orbit.plot()